# class Expressions: examples of use of each function

This webpage is for programmers who need examples of use of the functions of the class. The examples are designed to illustrate the syntax. They do not correspond to any meaningful model. For examples of models, visit  [biogeme.epfl.ch](http://biogeme.epfl.ch).

In [1]:
import datetime
print(datetime.datetime.now())

2019-09-25 14:00:07.839268


In [2]:
import biogeme.version as ver
print(ver.getText())

biogeme 3.2.2 [September 25, 2019]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import numpy as np
import pandas as pd
import biogeme.expressions as ex
import biogeme.database as db

We first create a small database

In [4]:
df = pd.DataFrame({'Person':[1,1,1,2,2],
                   'Exclude':[0,0,1,0,1],
                   'Variable1':[10,20,30,40,50],
                   'Variable2':[100,200,300,400,500],
                   'Choice':[1,2,3,1,2],
                   'Av1':[0,1,1,1,1],
                   'Av2':[1,1,1,1,1],
                   'Av3':[0,1,1,1,1]})
myData = db.Database('test',df)

The following type of expression is a literal called Variable that corresponds to an entry in the database.

In [5]:
Person=ex.Variable('Person')
Variable1=ex.Variable('Variable1')
Variable2=ex.Variable('Variable2')
Choice=ex.Variable('Choice')
Av1=ex.Variable('Av1')
Av2=ex.Variable('Av2')
Av3=ex.Variable('Av3')

It is possible to add a new column to thre database, that creates a new variable that can be used in expressions.

In [6]:
newvar = ex.DefineVariable('newvar',Variable1+Variable2,myData)
print(myData)

biogeme database test:
   Person  Exclude  Variable1  Variable2  Choice  Av1  Av2  Av3  newvar
0       1        0         10        100       1    0    1    0     110
1       1        0         20        200       2    1    1    1     220
2       1        1         30        300       3    1    1    1     330
3       2        0         40        400       1    1    1    1     440
4       2        1         50        500       2    1    1    1     550


The following type of expression is another literal, corresponding to an unknown parameter. 

In [7]:
beta1 = ex.Beta('beta1',0,None,None,0)
beta2 = ex.Beta('beta2',0,None,None,0)
beta3 = ex.Beta('beta3',1,None,None,1)
beta4 = ex.Beta('beta4',0,None,None,1)

Arithmetic operators are overloaded to allow standard manipulations of expressions. The first expression is $$e_1 = 2  \beta_1 - \frac{\exp(-\beta_2)}{\beta_3 (\beta_2 \geq \beta_1)},$$
where $(\beta_2 \geq \beta_1)$ equals 1 if $\beta_2 \geq \beta_1$ and 0 otherwise.

In [8]:
expr1 = 2 * beta1 - ex.exp(-beta2) / (beta3 * (beta2 >= beta1))
print(expr1)

((`2` * beta1(0)) - (exp((-beta2(0))) / (beta3(1) * (beta2(0) >= beta1(0)))))


The evaluation of expressions can be done in two ways. For simple expressions, the fonction getValue(), implemented in Python, returns the value of the expression.  

In [9]:
expr1.getValue()

-1.0

It is possible to modify the values of the parameters

In [10]:
newvalues = {'beta1':1,'beta2':2,'beta3':3,'beta4':2}
expr1.changeInitValues(newvalues)
expr1.getValue()

1.954888238921129

The function getValue_c() is implemented in C++, and works for any expression. It requires a database as input, and evaluates the expression for each entry in the database.
In the following example, as no variable of the database is involved in the expression, the output of the expression is the same for each entry.

In [11]:
expr1.getValue_c(myData)

[1.954888238921129,
 1.954888238921129,
 1.954888238921129,
 1.954888238921129,
 1.954888238921129]

The following function scans the expression and extracts a dict with all free parameters.

In [12]:
expr1.setOfBetas()

{'beta1', 'beta2'}

Options can be set to extract free parameters, fixed parameters, or both. 

In [13]:
expr1.setOfBetas(free=False,fixed=True)

{'beta3'}

In [14]:
expr1.setOfBetas(free=True,fixed=True)

{'beta1', 'beta2', 'beta3'}

In [15]:
expr1.getElementaryExpression('beta2')

beta2(2)

Let's consider an expression involving two variables $V_1$ and $V_2$: $$e_2 =2 \beta_1  V_1 - \frac{\exp(-\beta_2 V_2) }{ \beta_3  (\beta_2 \geq \beta_1)}.$$ Note that, in our example, the second term is numerically negligible with respect to the first one.

In [16]:
expr2 = 2 * beta1 * Variable1 - ex.exp(-beta2*Variable2) / (beta3 * (beta2 >= beta1))
print(expr2)

(((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / (beta3(3) * (beta2(2) >= beta1(1)))))


It is not a simple expression anymore, and only the function getValue_c can be invoked.

In [17]:
expr2.getValue_c(myData)

[20.0, 40.0, 60.0, 80.0, 100.0]

The following function extracts the names of the parameters apprearing in the expression

In [18]:
expr2.setOfBetas(free=True,fixed=True)

{'beta1', 'beta2', 'beta3'}

The list of parameters can also be obtained in the form of a dictionary.

In [19]:
expr2.dictOfBetas(free=True,fixed=True)

{'beta1': beta1(1), 'beta2': beta2(2), 'beta3': beta3(3)}

The list of variables can also be obtained in the form of a dictionary

In [20]:
expr2.dictOfVariables()

{'Variable1': Variable1, 'Variable2': Variable2}

or a set...

In [21]:
expr2.setOfVariables()

{'Variable1', 'Variable2'}

Expressions are defined recursively, using a tree representation. The following function describes the type of the upper most node of the tree.

In [22]:
expr2.getClassName()

'Minus'

The signature is a formal representation of the expression, assigning identifiers to each node of the tree, and representing them starting from the leaves. It is easy to parse, and is passed to the C++ implementation. 

In [23]:
expr2.getSignature()

[b'<Numeric>{4767305488},2',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<Times>{4767305424}(2),4767305488,4766965584',
 b'<Variable>{4766887824}"Variable1",5,2',
 b'<Times>{4767305552}(2),4767305424,4766887824',
 b'<Beta>{4766965712}"beta2"[0],1,1',
 b'<UnaryMinus>{4767305616}(1),4766965712',
 b'<Variable>{4492620496}"Variable2",6,3',
 b'<Times>{4767305680}(2),4767305616,4492620496',
 b'<exp>{4767309904}(1),4767305680',
 b'<Beta>{4766941840}"beta3"[1],2,0',
 b'<Beta>{4766965712}"beta2"[0],1,1',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<GreaterOrEqual>{4767309968}(2),4766965712,4766965584',
 b'<Times>{4767310032}(2),4766941840,4767309968',
 b'<Divide>{4767310096}(2),4767309904,4767310032',
 b'<Minus>{4767310160}(2),4767305552,4767310096']

The elementary expressions are
- free parameters,
- fixed parameters,
- random variables (for numerical integration),
- draws (for Monte-Carlo integration), and
- variables from the database.

The following function extracts all elementary expressions from a list of formulas, give them a unique numbering, and return them organized by group, as defined above (with the exception of the variables, that are directly available in the database).

In [24]:
collectionOfFormulas = [expr1,expr2]
elementaryExpressionIndex,allFreeBetas,freeBetaNames,allFixedBetas,fixedBetaNames,allRandomVariables,randomVariableNames,allDraws,drawNames = ex.defineNumberingOfElementaryExpressions(collectionOfFormulas,list(myData.data.columns))


Unique numbering for all elementary expressions

In [25]:
elementaryExpressionIndex

{'beta1': 0,
 'beta2': 1,
 'beta3': 2,
 'Person': 3,
 'Exclude': 4,
 'Variable1': 5,
 'Variable2': 6,
 'Choice': 7,
 'Av1': 8,
 'Av2': 9,
 'Av3': 10,
 'newvar': 11}

In [26]:
allFreeBetas

{'beta1': beta1(1), 'beta2': beta2(2)}

Each elementary expression has two ids. One unique across all elementary expressions, and one unique within each specific group

In [27]:
[(i.uniqueId,i.betaId) for k,i in allFreeBetas.items()]

[(0, 0), (1, 1)]

In [28]:
freeBetaNames

['beta1', 'beta2']

In [29]:
allFixedBetas

{'beta3': beta3(3)}

In [30]:
[(i.uniqueId,i.betaId) for k,i in allFixedBetas.items()]

[(2, 0)]

In [31]:
fixedBetaNames

['beta3']

In [32]:
allRandomVariables

{}

Monte Carlo integration is based on draws. 

In [33]:
myDraws = ex.bioDraws('myDraws','UNIFORM')
expr3 = ex.MonteCarlo(myDraws*myDraws)

In [34]:
print(expr3)

MonteCarlo((bioDraws('myDraws','UNIFORM') * bioDraws('myDraws','UNIFORM')))


Note that draws are not random variables, used for numerical integration.

In [35]:
expr3.dictOfRandomVariables()

{}

The following function reports the draws involved in an expression.

In [36]:
expr3.dictOfDraws()

{'myDraws': 'UNIFORM'}

The expression is a Monte-Carlo integration.

In [37]:
expr3.getClassName()

'MonteCarlo'

Here is its value. It is an approximation of $\int_0^1 x^2 dx=\frac{1}{3}$.

In [38]:
expr3.getValue_c(myData,numberOfDraws=100000)

[0.33374563323720036,
 0.33477803853019206,
 0.3322227141536586,
 0.3333700230283701,
 0.3329374854409177]

Here is its signature.

In [39]:
expr3.getSignature()

[b'<bioDraws>{4767437968}"myDraws",0,0',
 b'<bioDraws>{4767437968}"myDraws",0,0',
 b'<Times>{4767437904}(2),4767437968,4767437968',
 b'<MonteCarlo>{4767438160}(1),4767437904']

The same integral can be calculated using numerical integration, declaring a random variable. 

In [40]:
omega = ex.RandomVariable('omega')

Numerical integration calculates integrals between $-\infty$ and $+\infty$. Here, the interval being $[0,1]$, a change of variables is required.

In [41]:
a = 0
b = 1
x = a + (b-a) / ( 1 + ex.exp(-omega))
dx = (b-a) * ex.exp(-omega) * (1+ex.exp(-omega))**(-2) 
integrand = x * x
expr4 = ex.Integrate(integrand * dx /(b-a),'omega')

In this case, omega is a random variable.

In [42]:
expr4.dictOfRandomVariables()

{'omega': omega}

In [43]:
print(expr4)

Integrate(((((`0` + (`1` / (`1` + exp((-omega))))) * (`0` + (`1` / (`1` + exp((-omega)))))) * ((`1` * exp((-omega))) * ((`1` + exp((-omega))) ** `-2`))) / `1`),'omega')


Calculating its value requires the C++ implementation.

In [44]:
expr4.getValue_c(myData)

[0.3333323120662823,
 0.3333323120662823,
 0.3333323120662823,
 0.3333323120662823,
 0.3333323120662823]

We illustrate now the Elem function. It takes two arguments: a dictionary, and a formula for the key. For each entry in the database, the formula is evaluated, and its result identifies which formula in the dictionary should be evaluated.
Here is 'Person' is 1, the expression is $$e_1=2  \beta_1 - \frac{\exp(-\beta_2)}{\beta_3 (\beta_2 \geq \beta_1)},$$ and if 'Person' is 2, the expression is $$e_2=2 \beta_1  V_1 - \frac{\exp(-\beta_2 V_2) }{ \beta_3  (\beta_2 \geq \beta_1)}.$$ As it is a regular expression, it can be included in any formula. Here, we illustrate it by dividing the result by 10.

In [45]:
elemExpr = ex.Elem({1:expr1,2:expr2},Person) 
expr5 =  elemExpr / 10
print(expr5)

({{1:((`2` * beta1(1)) - (exp((-beta2(2))) / (beta3(3) * (beta2(2) >= beta1(1))))),2:(((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / (beta3(3) * (beta2(2) >= beta1(1)))))}[Person] / `10`)


In [46]:
expr5.dictOfVariables()

{'Variable1': Variable1, 'Variable2': Variable2, 'Person': Person}

In [47]:
expr5.getValue_c(myData)

[0.19548882389211292, 0.19548882389211292, 0.19548882389211292, 8.0, 10.0]

The next expression is simply the sum of multiples expressions. The argument is a list of expressions. 

In [48]:
expr6 = ex.bioMultSum([expr1,expr2,expr4])

In [49]:
print(expr6)

bioMultSum(((`2` * beta1(1)) - (exp((-beta2(2))) / (beta3(3) * (beta2(2) >= beta1(1))))) + (((`2` * beta1(1)) * Variable1) - (exp(((-beta2(2)) * Variable2)) / (beta3(3) * (beta2(2) >= beta1(1))))) + Integrate(((((`0` + (`1` / (`1` + exp((-omega))))) * (`0` + (`1` / (`1` + exp((-omega)))))) * ((`1` * exp((-omega))) * ((`1` + exp((-omega))) ** `-2`))) / `1`),'omega'))


In [50]:
expr6.getValue_c(myData,100000)

[22.28822055098741,
 42.28822055098741,
 62.28822055098741,
 82.2882205509874,
 102.2882205509874]

We now illustrate how to calculate a logit model, that is $$ \frac{y_1 e^{V_1}}{y_0 e^{V_0}+y_1 e^{V_1}+y_2 e^{V_2}}$$ where $V_0=-\beta_1$, $V_1=-\beta_2$ and $V_2=-\beta_1$, and $y_i = 1$, $i=1,2,3$.

In [51]:
V = {0:-beta1,1:-beta2,2:-beta1}
av = {0:1,1:1,2:1}
expr7 = ex.LogLogit(V,av,1)

In [52]:
expr7.getValue()

-1.861994804058251

It is actually better to use the C++ implementation using the following syntax.

In [53]:
expr8 = ex.bioLogLogit(V,av,1)

In [54]:
expr8.getValue_c(myData)

[-1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512]

As the result is a numpy array, it can be used for any calculation. Here, we show how to calculate the logsum

In [55]:
for v in V.values():
    print(v.getValue_c(myData))

[-1.0, -1.0, -1.0, -1.0, -1.0]
[-2.0, -2.0, -2.0, -2.0, -2.0]
[-1.0, -1.0, -1.0, -1.0, -1.0]


In [56]:
logsum = np.log(np.sum([np.exp(v.getValue_c(myData)) for v in V.values()],axis=1))
logsum

array([ 0.60943791, -0.39056209,  0.60943791])

When all the alternatives are always available, there is a specific version of the logit formula for that

In [57]:
expr8bis = ex.bioLogLogitFullChoiceSet(V,av=None,choice=1)
expr8bis.getValue_c(myData)

[-1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512,
 -1.8619948040582512]

It is possible to calculate the derivative of a formula with respect to a literal: $$e_9=\frac{\partial e_8}{\partial \beta_2}.$$

In [58]:
expr9 = ex.Derive(expr8,'beta2')

In [59]:
expr9.getValue_c(myData)

[-0.8446375965030364,
 -0.8446375965030364,
 -0.8446375965030364,
 -0.8446375965030364,
 -0.8446375965030364]

Biogeme also provides an approximation of the CDF of the normal distribution: $$e_{10}= \frac{1}{{\sigma \sqrt {2\pi } }}\int_{-\infty}^t e^{{{ - \left( {x - \mu } \right)^2 } \mathord{\left/ {\vphantom {{ - \left( {x - \mu } \right)^2 } {2\sigma ^2 }}} \right. } {2\sigma ^2 }}}dx$$

In [60]:
expr10 = ex.bioNormalCdf(Variable1/10-1)

In [61]:
expr10.getValue_c(myData)

[0.5,
 0.8413447460685283,
 0.9772498680518218,
 0.99865010196837,
 0.9999683287581669]

Min and max operators are also available. To avoid any ambiguity with the Python operator, they are called bioMin and bioMax. 

In [62]:
expr11 = ex.bioMin(expr5,expr10)
expr11.getValue_c(myData)

[0.19548882389211292,
 0.19548882389211292,
 0.19548882389211292,
 0.99865010196837,
 0.9999683287581669]

In [63]:
expr12 = ex.bioMax(expr5,expr10)
expr12.getValue_c(myData)

[0.5, 0.8413447460685283, 0.9772498680518218, 8.0, 10.0]

For the sake of efficiency, it is possible to specify explicitly a linear function, where each term is the product of a parameter and a variable.

In [64]:
terms = [(beta1,ex.Variable('Variable1')),(beta2,ex.Variable('Variable2')),(beta3,ex.Variable('newvar'))]

In [65]:
expr13 = ex.bioLinearUtility(terms)


In [66]:
expr13.getValue_c(myData)

[540.0, 1080.0, 1620.0, 2160.0, 2700.0]

In terms of specification, it is equivalent to the expression below. But the calculation of the derivates is more efficient, as the linear structure of the specification is exploited.

In [67]:
expr13bis = beta1 * Variable1 + beta2 * Variable2 + beta3 * newvar

In [68]:
expr13bis.getValue_c(myData)

[540.0, 1080.0, 1620.0, 2160.0, 2700.0]

A Pythonic way to write a linear utility function

In [69]:
variables = ['v1','v2','v3','cost','time','headway']
coefficients = {f'{v}':ex.Beta(f'beta_{v}',0,None,None,0) for v in variables}
terms = [coefficients[v] * ex.Variable(v) for v in variables]
util = sum(terms)
print(util)

((((((`0` + (beta_v1(0) * v1)) + (beta_v2(0) * v2)) + (beta_v3(0) * v3)) + (beta_cost(0) * cost)) + (beta_time(0) * time)) + (beta_headway(0) * headway))


# Signatures

The Python library communicates the expressions to the C++ library using a syntax called a "signature". We describe and illustrate now the signature for each expression. Each expression is identified by an identifier provided by Python using the function 'id'. 

In [70]:
id(expr1)

4757417040

## Numerical expression

< Numeric >{identifier},value I DO NOT KNOW HOW TO GET RID OF THE WHITE SPACES

In [71]:
ex.Numeric(0).getSignature()

[b'<Numeric>{4767524816},0']

## Beta parameters

< Beta >{identifier}"name"[status],uniqueId,betaId'
where 
- status is 0 for free parameters, and non zero for fixed parameters,
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- betaId is a unique index given by Biogeme to all free parameters, and to all fixed parameters.

In [72]:
beta1.getSignature()

[b'<Beta>{4766965584}"beta1"[0],0,0']

In [73]:
beta3.getSignature()

[b'<Beta>{4766941840}"beta3"[1],2,0']

## Variables

< Variable >{identifier}"name",uniqueId,variableId 
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- variableId is a unique index given by Biogeme to all variables.


In [74]:
Variable1.getSignature()

[b'<Variable>{4766887824}"Variable1",5,2']

## Random variables

< RandomVariable >{identifier}"name",uniqueId,randomVariableId
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- randomVariableId is a unique index given by Biogeme to all random variables.

In [75]:
omega.getSignature()

[b'<RandomVariable>{4767440144}"omega",3,0']

## Draws

< bioDraws >{identifier}"name",uniqueId,drawId
where
- uniqueId is a unique index given by Biogeme to all elementary expressions,
- drawId is a unique index given by Biogeme to all draws.


In [76]:
myDraws.getSignature()

[b'<bioDraws>{4767437968}"myDraws",0,0']

## General expression

< operator >{identifier}(numberOfChildren),idFirstChild,idSecondChild,idThirdChild, etc...
where the number of identifiers given after the comma matches the reported number of children. 

Specific examples are reported below.

### Binary operator

< operator >{identifier}(2),idFirstChild,idSecondChild 
where operator is one of: 
    - 'Plus'
    - 'Minus'
    - 'Times'
    - 'Divide'
    - 'Power'
    - 'bioMin'
    - 'bioMax'
    - 'And'
    - 'Or'
    - 'Equal'
    - 'NotEqual'
    - 'LessOrEqual'
    - 'GreaterOrEqual'
    - 'Less'
    - 'Greater'


In [77]:
sum = beta1 + Variable1

In [78]:
sum.getSignature()

[b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<Variable>{4766887824}"Variable1",5,2',
 b'<Plus>{4767312592}(2),4766965584,4766887824']

### Unary operator

< operator >{identifier}(1),idChild, 
where operator is one of: 
    - 'UnaryMinus'
    - 'MonteCarlo'
    - 'bioNormalCdf'
    - 'PanelLikelihoodTrajectory'
    - 'exp'
    - 'log'

In [79]:
m = -beta1

In [80]:
m.getSignature()

[b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<UnaryMinus>{4767132560}(1),4766965584']

## LogLogit

< LogLogit >{identifier}(nbrOfAlternatives),chosenAlt,altNumber,utility,availability,altNumber,utility,availability, etc.

In [81]:
expr7.getSignature()

[b'<Numeric>{4767356496},1',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<UnaryMinus>{4767133072}(1),4766965584',
 b'<Beta>{4766965712}"beta2"[0],1,1',
 b'<UnaryMinus>{4767336400}(1),4766965712',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<UnaryMinus>{4767336208}(1),4766965584',
 b'<Numeric>{4757415824},1',
 b'<Numeric>{4767334992},1',
 b'<Numeric>{4767335184},1',
 b'<LogLogit>{4767335120}(3),4767356496,0,4767133072,4757415824,1,4767336400,4767334992,2,4767336208,4767335184']

## Derive

< Derive >{identifier},id of expression to derive,unique index of elementary expression

In [82]:
expr9.getSignature()

[b'<Numeric>{4767460624},1',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<UnaryMinus>{4767133072}(1),4766965584',
 b'<Beta>{4766965712}"beta2"[0],1,1',
 b'<UnaryMinus>{4767336400}(1),4766965712',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<UnaryMinus>{4767336208}(1),4766965584',
 b'<Numeric>{4757392784},1',
 b'<Numeric>{4767458128},1',
 b'<Numeric>{4767457680},1',
 b'<bioLogLogit>{4767457552}(3),4767460624,0,4767133072,4757392784,1,4767336400,4767458128,2,4767336208,4767457680',
 b'<Derive>{4804333840},4767457552,1']

## Integrate

< Integrate >{identifier},id of expression to derive,index of random variable

In [83]:
expr4.getSignature()

[b'<Numeric>{4767310224},0',
 b'<Numeric>{4767313232},1',
 b'<Numeric>{4767312400},1',
 b'<RandomVariable>{4767440144}"omega",3,0',
 b'<UnaryMinus>{4767336528}(1),4767440144',
 b'<exp>{4767313872}(1),4767336528',
 b'<Plus>{4767313488}(2),4767312400,4767313872',
 b'<Divide>{4767312848}(2),4767313232,4767313488',
 b'<Plus>{4767312784}(2),4767310224,4767312848',
 b'<Numeric>{4767310224},0',
 b'<Numeric>{4767313232},1',
 b'<Numeric>{4767312400},1',
 b'<RandomVariable>{4767440144}"omega",3,0',
 b'<UnaryMinus>{4767336528}(1),4767440144',
 b'<exp>{4767313872}(1),4767336528',
 b'<Plus>{4767313488}(2),4767312400,4767313872',
 b'<Divide>{4767312848}(2),4767313232,4767313488',
 b'<Plus>{4767312784}(2),4767310224,4767312848',
 b'<Times>{4767441552}(2),4767312784,4767312784',
 b'<Numeric>{4767310736},1',
 b'<RandomVariable>{4767440144}"omega",3,0',
 b'<UnaryMinus>{4767338192}(1),4767440144',
 b'<exp>{4767310672}(1),4767338192',
 b'<Times>{4767311504}(2),4767310736,4767310672',
 b'<Numeric>{47673133

## Elem

< Elem >{identifier}(numberOfExpressions),keyId,value1,expression1,value2,expression2, etc...

where
- keyId is the identifier of the expression calculating the key,
- the number of pairs valuex,expressionx must correspond to the value of numberOfExpressions

In [84]:
elemExpr.getSignature()

[b'<Variable>{4766887888}"Person",3,0',
 b'<Numeric>{4492714704},2',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<Times>{4492717008}(2),4492714704,4766965584',
 b'<Beta>{4766965712}"beta2"[0],1,1',
 b'<UnaryMinus>{4757416976}(1),4766965712',
 b'<exp>{4753081808}(1),4757416976',
 b'<Beta>{4766941840}"beta3"[1],2,0',
 b'<Beta>{4766965712}"beta2"[0],1,1',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<GreaterOrEqual>{4761438608}(2),4766965712,4766965584',
 b'<Times>{4757414544}(2),4766941840,4761438608',
 b'<Divide>{4757415120}(2),4753081808,4757414544',
 b'<Minus>{4757417040}(2),4492717008,4757415120',
 b'<Numeric>{4767305488},2',
 b'<Beta>{4766965584}"beta1"[0],0,0',
 b'<Times>{4767305424}(2),4767305488,4766965584',
 b'<Variable>{4766887824}"Variable1",5,2',
 b'<Times>{4767305552}(2),4767305424,4766887824',
 b'<Beta>{4766965712}"beta2"[0],1,1',
 b'<UnaryMinus>{4767305616}(1),4766965712',
 b'<Variable>{4492620496}"Variable2",6,3',
 b'<Times>{4767305680}(2),4767305616,4492620496',
 b'<exp>{476730